### 역발상 종목 선별

In [3]:
import sys
import pandas as pd
from tqdm import tqdm
sys.path.append('../')
from stock_screener.data_reader import *

In [4]:
# 최근 거래일자 획득
date = get_recent_biz_day()

#해당일의 종목별 지표 정보 획득
per_pbr_divdend_data = get_per_pbr_dividend(date)

UnboundLocalError: local variable 'df' referenced before assignment

In [3]:
# 종목의 업종분류 획득 (WICS)
%time sector_data = get_sector(date)

CPU times: user 246 ms, sys: 36.3 ms, total: 282 ms
Wall time: 15.3 s


In [4]:
# 업종별로 PER / PBR / 배당수익률 정보 join
sector_data = {k:pd.merge(v, 
                          per_pbr_divdend_data, 
                          on=['종목명','종목코드']).sort_values('PER')
                for k,v in sector_data.items()}

In [5]:
sector_code_name = {k: ' '.join(v['섹터명'].iloc[0].split()[1:]) \
                    for k,v in sector_data.items()}

In [6]:
# PER 평균값 데이터
per_mean = dict()

# 시장 PER 평균
per_mean['전체'] = per_pbr_divdend_data['PER'].mean()

# 업종별 PER 평균
for k,v in sector_data.items():
    per_mean[sector_code_name[k]] = v['PER'].mean()

In [7]:
per_mean_df = pd.DataFrame(per_mean.items())
per_mean_df.columns = ['업종', '평균PER']

In [8]:
per_mean_df.sort_values('평균PER')

,업종,평균PER
16,은행,4.152222
17,증권,7.980000
18,다각화된금융,12.805000
28,유틸리티,14.353529
19,보험,16.777500
20,부동산,18.520000
4,상업서비스와공급품,20.272353
5,운송,24.040909
11,식품과기본식료품소매,25.093333
13,가정용품과개인용품,28.802500


In [9]:
def pass_criteria(code):

    # 재무제표 데이터에서 원하는 조건을 만족하는 경우 True, 아니면 False 반환
    annual_data, quarter_data = get_financial_summary(code)

    try:
        # 최근 3개년 영업이익 흑자
        num_year = 3
        annual_profit = annual_data['영업이익'].tail(num_year)
        if not all(x > 0 for x in annual_profit):
            return False

        # 최근 3개년 ROE >= 8
        num_year = 3
        annual_roe = annual_data['ROE'].tail(num_year)
        if not all(x >= 8 for x in annual_roe):
            return False

        # 최근 3개년 배당수익률 >= 2
        num_year = 3
        annual_dvd = annual_data['배당수익률'].tail(num_year)
        if not all(x >= 2 for x in annual_dvd):
            return False

        # 최근 4분기 영업이익 흑자
        num_quarter = 4
        quarter_profit = quarter_data['영업이익'].tail(num_quarter)
        if not all(x > 0 for x in quarter_profit):
            return False
    except:
        # 체크하려는 값이 nan인 경우거나, 데이터가 불충분한 경우
        return False

    return True


In [10]:
# 기준값 만족하는 종목을 업종별로 스크리닝
corp_by_sector = dict()

for sector_code, sector_df in tqdm(sector_data.items()):

    sector_name = sector_code_name[sector_code]
    cutoff_per = sector_df['PER'].quantile(0.5)
    
    # PER이 낮은 종목 -> 높은 종목 순으로 체크
    for _, x in sector_df.iterrows():

        if pass_criteria(x['종목코드']):
            corp_by_sector[sector_name] = dict(x)
            break

        # 업종 PER의 중위수보다 작은 PER을 갖는 종목만 고려한다
        elif x['PER'] >= cutoff_per:
            break

    # 적절한 종목이 없는 경우, 빈값을 채운다
    if sector_name not in corp_by_sector:
        corp_by_sector[sector_name] = {'기준일':x['기준일']}

100%|██████████| 28/28 [01:51<00:00,  3.97s/it]


In [11]:
len(corp_by_sector)

28

In [12]:
corp_by_sector_df = pd.DataFrame(corp_by_sector.values())
corp_by_sector_df['업종'] = corp_by_sector.keys()
corp_by_sector_df['업종평균PER'] = [per_mean[k] for k in corp_by_sector]
corp_by_sector_df = corp_by_sector_df[['기준일', '업종', '종목명', '종목코드', '종가', 
                                       'PER', 'PBR', '배당수익률', '업종평균PER']]
corp_by_sector_df.fillna('', inplace=True)

In [13]:
corp_by_sector_df

,기준일,업종,종목명,종목코드,종가,PER,PBR,배당수익률,업종평균PER
0,20201113,에너지,극동유화,014530,3395,8.05,0.7,4.71,97.850000
1,20201113,소재,케이디켐,221980,12600,6.57,0.66,3.97,40.015549
2,20201113,자본재,삼호개발,010960,4130,4.55,0.54,4.6,43.584677
3,20201113,상업서비스와공급품,고려신용정보,049720,5500,11.55,3.49,4.55,20.272353
4,20201113,운송,KSS해운,044450,10800,10.44,0.9,2.5,24.040909
5,20201113,자동차와부품,,,,,,,53.412778
6,20201113,내구소비재와의류,쿠쿠홀딩스,192400,96000,7.51,1.01,3.13,58.276747
7,20201113,"호텔,레스토랑,레저등",,,,,,,509.983571
8,20201113,소매(유통),GS홈쇼핑,028150,139300,7.6,0.8,4.67,74.660000
9,20201113,교육서비스,디지털대성,068930,7480,9.82,2.03,4.01,41.013750
